# AIS Forecasting - Model Development

This notebook covers the model development pipeline for AIS trajectory forecasting.

## Contents
1. Setup and Configuration
2. Data Loading
3. Model Configuration
4. Training Pipeline
5. Model Comparison
6. Hyperparameter Tuning
7. Model Evaluation
8. Save Models

## 1. Setup and Configuration

In [ ]:
import os
import sys
import warnings
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pytorch_lightning as pl
from pathlib import Path

# Add src to path
project_root = Path().absolute().parent
sys.path.append(str(project_root / 'src'))

from src.data.loader import AISDataLoader
from src.data.preprocessing import AISDataPreprocessor
from src.models.tft_model import TFTModel
from src.models.nbeats_model import NBeatsModel
from src.utils.metrics import calculate_metrics
from src.utils.optimize import OptunaOptimizer
from src.visualization.plots import plot_forecast, plot_training_history

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
pl.seed_everything(42)

print(f"Project root: {project_root}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load Configuration

In [ ]:
# Load default configuration
config_path = project_root / 'config' / 'default.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"- Data path: {config['data']['processed_path']}")
print(f"- Model output: {config['model']['output_dir']}")
print(f"- Sequence length: {config['model']['sequence_length']}")
print(f"- Prediction horizon: {config['model']['prediction_horizon']}")

## 3. Load Preprocessed Data

In [ ]:
# Initialize data loader and preprocessor
data_loader = AISDataLoader(config)
preprocessor = AISDataPreprocessor(config)

# Note: In a real scenario, you would load actual AIS data
# For demonstration, we'll create synthetic data
print("Loading preprocessed data...")

# Load or create sample data
try:
    # Try to load existing processed data
    train_data = pd.read_parquet(project_root / config['data']['processed_path'] / 'train.parquet')
    val_data = pd.read_parquet(project_root / config['data']['processed_path'] / 'val.parquet')
    test_data = pd.read_parquet(project_root / config['data']['processed_path'] / 'test.parquet')
    print(f"Loaded existing data: {len(train_data)} train, {len(val_data)} val, {len(test_data)} test samples")
except FileNotFoundError:
    print("No processed data found. Creating synthetic dataset for demonstration...")
    
    # Create synthetic AIS data
    n_vessels = 100
    n_timestamps = 1000
    
    vessels = [f"VESSEL_{i:03d}" for i in range(n_vessels)]
    timestamps = pd.date_range('2023-01-01', periods=n_timestamps, freq='1H')
    
    data = []
    for vessel in vessels:
        # Simulate vessel trajectory
        lat_base = np.random.uniform(40, 60)
        lon_base = np.random.uniform(-10, 10)
        
        for i, ts in enumerate(timestamps):
            # Add some trajectory variation
            lat = lat_base + 0.1 * np.sin(i * 0.01) + np.random.normal(0, 0.01)
            lon = lon_base + 0.1 * np.cos(i * 0.01) + np.random.normal(0, 0.01)
            speed = np.random.uniform(5, 25)
            heading = np.random.uniform(0, 360)
            
            data.append({
                'mmsi': vessel,
                'timestamp': ts,
                'latitude': lat,
                'longitude': lon,
                'speed': speed,
                'heading': heading
            })
    
    df = pd.DataFrame(data)
    
    # Split data
    train_size = int(0.7 * len(timestamps))
    val_size = int(0.15 * len(timestamps))
    
    train_data = df[df['timestamp'] < timestamps[train_size]]
    val_data = df[(df['timestamp'] >= timestamps[train_size]) & 
                  (df['timestamp'] < timestamps[train_size + val_size])]
    test_data = df[df['timestamp'] >= timestamps[train_size + val_size]]
    
    print(f"Created synthetic data: {len(train_data)} train, {len(val_data)} val, {len(test_data)} test samples")

print(f"\nData shapes:")
print(f"- Train: {train_data.shape}")
print(f"- Validation: {val_data.shape}")
print(f"- Test: {test_data.shape}")

## 4. Model Configuration and Training

In [ ]:
# Load experiment configurations
    "tft_config_path = project_root / 'config' / 'experiment_configs' / 'experiment_tft.yaml'
",
    "nbeats_config_path = project_root / 'config' / 'experiment_configs' / 'experiment_nbeats.yaml'
",
with open(tft_config_path, 'r') as f:
    tft_config = yaml.safe_load(f)

with open(nbeats_config_path, 'r') as f:
    nbeats_config = yaml.safe_load(f)

print("Model configurations loaded")
print(f"TFT config: {tft_config['model']['model_type']}")
print(f"N-BEATS config: {nbeats_config['model']['model_type']}")

### 4.1 Prepare Data for Training

In [ ]:
# Prepare time series sequences
def prepare_sequences(data, sequence_length, prediction_horizon):
    """Prepare time series sequences for training."""
    sequences = []
    targets = []
    
    # Group by vessel
    for mmsi, group in data.groupby('mmsi'):
        group = group.sort_values('timestamp').reset_index(drop=True)
        
        # Create sequences
        for i in range(len(group) - sequence_length - prediction_horizon + 1):
            seq = group.iloc[i:i+sequence_length][['latitude', 'longitude', 'speed', 'heading']].values
            target = group.iloc[i+sequence_length:i+sequence_length+prediction_horizon][['latitude', 'longitude']].values
            
            sequences.append(seq)
            targets.append(target)
    
    return np.array(sequences), np.array(targets)

sequence_length = config['model']['sequence_length']
prediction_horizon = config['model']['prediction_horizon']

print("Preparing sequences...")
X_train, y_train = prepare_sequences(train_data, sequence_length, prediction_horizon)
X_val, y_val = prepare_sequences(val_data, sequence_length, prediction_horizon)
X_test, y_test = prepare_sequences(test_data, sequence_length, prediction_horizon)

print(f"Sequence shapes:")
print(f"- X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"- X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"- X_test: {X_test.shape}, y_test: {y_test.shape}")

### 4.2 Train TFT Model

In [ ]:
print("Training TFT Model...")

# Initialize TFT model
tft_model = TFTModel(tft_config)

# Prepare data loaders
train_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_train), 
    torch.FloatTensor(y_train)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_val), 
    torch.FloatTensor(y_val)
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=tft_config['training']['batch_size'], 
    shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=tft_config['training']['batch_size'], 
    shuffle=False
)

# Setup trainer
trainer = pl.Trainer(
    max_epochs=tft_config['training']['epochs'],
    accelerator='auto',
    devices='auto',
    deterministic=True,
    enable_progress_bar=True,
    log_every_n_steps=10
)

# Train model
trainer.fit(tft_model, train_loader, val_loader)

print("TFT training completed!")

### 4.3 Train N-BEATS Model

In [ ]:
print("Training N-BEATS Model...")

# Initialize N-BEATS model
nbeats_model = NBeatsModel(nbeats_config)

# Prepare data loaders with N-BEATS batch size
train_loader_nbeats = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=nbeats_config['training']['batch_size'], 
    shuffle=True
)
val_loader_nbeats = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=nbeats_config['training']['batch_size'], 
    shuffle=False
)

# Setup trainer for N-BEATS
trainer_nbeats = pl.Trainer(
    max_epochs=nbeats_config['training']['epochs'],
    accelerator='auto',
    devices='auto',
    deterministic=True,
    enable_progress_bar=True,
    log_every_n_steps=10
)

# Train model
trainer_nbeats.fit(nbeats_model, train_loader_nbeats, val_loader_nbeats)

print("N-BEATS training completed!")

## 5. Model Evaluation and Comparison

In [ ]:
# Evaluate both models on test set
print("Evaluating models on test set...")

# Prepare test data
test_dataset = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_test), 
    torch.FloatTensor(y_test)
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=32, 
    shuffle=False
)

# Get predictions from both models
tft_predictions = trainer.predict(tft_model, test_loader)
nbeats_predictions = trainer_nbeats.predict(nbeats_model, test_loader)

# Convert to numpy arrays
tft_pred = torch.cat(tft_predictions).numpy()
nbeats_pred = torch.cat(nbeats_predictions).numpy()
y_true = y_test

print(f"Prediction shapes: TFT {tft_pred.shape}, N-BEATS {nbeats_pred.shape}, True {y_true.shape}")

In [ ]:
# Calculate metrics for both models
from src.utils.metrics import calculate_mae, calculate_rmse, calculate_smape

print("\n=== Model Comparison ===")
print("\nTFT Model:")
tft_mae = calculate_mae(y_true, tft_pred)
tft_rmse = calculate_rmse(y_true, tft_pred)
tft_smape = calculate_smape(y_true, tft_pred)
print(f"  MAE: {tft_mae:.4f}")
print(f"  RMSE: {tft_rmse:.4f}")
print(f"  SMAPE: {tft_smape:.4f}")

print("\nN-BEATS Model:")
nbeats_mae = calculate_mae(y_true, nbeats_pred)
nbeats_rmse = calculate_rmse(y_true, nbeats_pred)
nbeats_smape = calculate_smape(y_true, nbeats_pred)
print(f"  MAE: {nbeats_mae:.4f}")
print(f"  RMSE: {nbeats_rmse:.4f}")
print(f"  SMAPE: {nbeats_smape:.4f}")

# Determine best model
best_model = "TFT" if tft_mae < nbeats_mae else "N-BEATS"
print(f"\nBest model (by MAE): {best_model}")

## 6. Visualization

In [ ]:
# Plot sample predictions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Model Predictions Comparison', fontsize=16)

# Select a few samples for visualization
sample_indices = [0, 5, 10, 15]

for idx, sample_idx in enumerate(sample_indices):
    if sample_idx >= len(y_true):
        continue
        
    row = idx // 2
    col = idx % 2
    
    ax = axes[row, col]
    
    # Plot true trajectory
    ax.plot(y_true[sample_idx, :, 1], y_true[sample_idx, :, 0], 'ko-', label='True', linewidth=2)
    
    # Plot TFT prediction
    ax.plot(tft_pred[sample_idx, :, 1], tft_pred[sample_idx, :, 0], 'ro-', label='TFT', alpha=0.7)
    
    # Plot N-BEATS prediction
    ax.plot(nbeats_pred[sample_idx, :, 1], nbeats_pred[sample_idx, :, 0], 'bo-', label='N-BEATS', alpha=0.7)
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'Sample {sample_idx + 1}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Save Models

In [ ]:
# Create models directory
models_dir = project_root / 'models'
models_dir.mkdir(exist_ok=True)

# Save TFT model
tft_path = models_dir / 'tft_model.ckpt'
trainer.save_checkpoint(tft_path)
print(f"TFT model saved to: {tft_path}")

# Save N-BEATS model
nbeats_path = models_dir / 'nbeats_model.ckpt'
trainer_nbeats.save_checkpoint(nbeats_path)
print(f"N-BEATS model saved to: {nbeats_path}")

# Save metrics comparison
metrics_comparison = {
    'TFT': {
        'MAE': float(tft_mae),
        'RMSE': float(tft_rmse),
        'SMAPE': float(tft_smape)
    },
    'N-BEATS': {
        'MAE': float(nbeats_mae),
        'RMSE': float(nbeats_rmse),
        'SMAPE': float(nbeats_smape)
    },
    'best_model': best_model
}

import json
with open(models_dir / 'metrics_comparison.json', 'w') as f:
    json.dump(metrics_comparison, f, indent=2)

print("\nModel development completed!")
print(f"Models and metrics saved to: {models_dir}")